# Baseline

# L2 正则 weight_decay=1 欠拟合

## Baseline思路

将不定长字符转换为定长字符的识别问题，并使用CNN完成训练和验证，具体包括以下几个步骤：     
                   
- 赛题数据读取（封装为Pytorch的Dataset和DataLoder）
             
- 构建CNN模型（使用Pytorch搭建）
              
- 模型训练与验证
                
- 模型结果预测


##  运行环境及安装示例   
- 运行环境要求：Python2/3，Pytorch1.x，内存4G，有无GPU都可以。         
                        
下面给出python3.7+ torch1.3.1gpu版本的环境安装示例：      
                               
- 首先在Anaconda中创建一个专门用于本次天池练习赛的虚拟环境。          

>$conda create -n py37_torch131 python=3.7      
                                
- 激活环境，并安装pytorch1.3.1
                                     
>$source activate py37_torch131                         

>$conda install pytorch=1.3.1 torchvision cudatoolkit=10.0                     
       
       
- 通过下面的命令一键安装所需其它依赖库     

>$pip install jupyter tqdm opencv-python matplotlib pandas                                  
       
- 启动notebook，即可开始baseline代码的学习                  

>$jupyter-notebook    

## 导入常用的包

In [1]:
import os, sys, glob, shutil, json
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

## 步骤1：定义读取图像的Dataset    

In [2]:
class SVHNDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label 
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None

    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')

        if self.transform is not None:
            img = self.transform(img)
        
        # 设置最长的字符长度为5个
        lbl = np.array(self.img_label[index], dtype=np.int)
        lbl = list(lbl)  + (5 - len(lbl)) * [10]
        return img, torch.from_numpy(np.array(lbl[:5]))

    def __len__(self):
        return len(self.img_path)

## 步骤2：定义训练数据和验证数据的Dataset 

In [3]:
train_path = glob.glob('../../data/Datawhale-Tianchi-CV_SVHN/mchar_train/*.png')
train_path.sort()
train_json = json.load(open('../../data/Datawhale-Tianchi-CV_SVHN/mchar_train.json'))
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))

train_loader = torch.utils.data.DataLoader(
    SVHNDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    transforms.ColorJitter(0.3, 0.3, 0.2),
                    transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=800, #默认40 
    shuffle=True, 
    num_workers=0, # win下num_works改为0,linux=10
)

val_path = glob.glob('../../data/Datawhale-Tianchi-CV_SVHN/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('../../data/Datawhale-Tianchi-CV_SVHN/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

val_loader = torch.utils.data.DataLoader(
    SVHNDataset(val_path, val_label,
                transforms.Compose([
                    transforms.Resize((60, 120)),
                    # transforms.Resize((64, 128)),
                    # transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=800, #默认40 
    shuffle=False, 
    num_workers=0, # win下num_works改为0,linux=10
)

30000 30000
10000 10000


## 步骤3：定义字符分类模型
使用resnet18的模型作为特征提取模块

In [4]:
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()
                
        model_conv = models.resnet18(pretrained=True)
        model_conv.avgpool = nn.AdaptiveAvgPool2d(1)
        model_conv = nn.Sequential(*list(model_conv.children())[:-1])
        self.cnn = model_conv
        
        self.fc1 = nn.Linear(512, 11)
        self.fc2 = nn.Linear(512, 11)
        self.fc3 = nn.Linear(512, 11)
        self.fc4 = nn.Linear(512, 11)
        self.fc5 = nn.Linear(512, 11)
    
    def forward(self, img):        
        feat = self.cnn(img)
        # print(feat.shape)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        return c1, c2, c3, c4, c5

## 步骤4：定义训练、验证和预测模块 

In [5]:
def train(train_loader, model, criterion, optimizer):
    # 切换模型为训练模式
    model.train()
    train_loss = []
    
    for i, (input, target) in enumerate(train_loader):
        if use_cuda:
            input = input.cuda()
            target = target.cuda()
            
        c0, c1, c2, c3, c4 = model(input)
        target = target.long() #添加
        loss = criterion(c0, target[:, 0]) + \
                criterion(c1, target[:, 1]) + \
                criterion(c2, target[:, 2]) + \
                criterion(c3, target[:, 3]) + \
                criterion(c4, target[:, 4])
        
        # loss /= 6
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # if i % 100 == 0:
            # print(loss.item())
        
        train_loss.append(loss.item())
    return np.mean(train_loss)

def validate(val_loader, model, criterion):
    # 切换模型为预测模型
    model.eval()
    val_loss = []

    # 不记录模型梯度信息
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            if use_cuda:
                input = input.cuda()
                target = target.cuda()
            
            c0, c1, c2, c3, c4 = model(input)
            target = target.long() #添加
            loss = criterion(c0, target[:, 0]) + \
                    criterion(c1, target[:, 1]) + \
                    criterion(c2, target[:, 2]) + \
                    criterion(c3, target[:, 3]) + \
                    criterion(c4, target[:, 4])
            # loss /= 6
            val_loss.append(loss.item())
    return np.mean(val_loss)

def predict(test_loader, model, tta=10):
    model.eval()
    test_pred_tta = None
    
    # TTA 次数
    for _ in range(tta):
        test_pred = []
    
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                if use_cuda:
                    input = input.cuda()
                
                c0, c1, c2, c3, c4 = model(input)
                if use_cuda:
                    output = np.concatenate([
                        c0.data.cpu().numpy(), 
                        c1.data.cpu().numpy(),
                        c2.data.cpu().numpy(), 
                        c3.data.cpu().numpy(),
                        c4.data.cpu().numpy()], axis=1)
                else:
                    output = np.concatenate([
                        c0.data.numpy(), 
                        c1.data.numpy(),
                        c2.data.numpy(), 
                        c3.data.numpy(),
                        c4.data.numpy()], axis=1)
                
                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

## 步骤5：迭代训练和验证模型

In [6]:
model = SVHN_Model1()
criterion = nn.CrossEntropyLoss()
wd = 1
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=wd)
best_loss = 1000.0
best_acc = 0

use_cuda = True
if use_cuda:
    model = model.cuda()


loss_increase_num = 0 # 用于调节学习率
val_loss_last = best_loss # 上一次的验证集误差

for epoch in range(20):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_loss = validate(val_loader, model, criterion)
    
    val_label = [''.join(map(str, x)) for x in val_loader.dataset.img_label]
    val_predict_label = predict(val_loader, model, 1)
    val_predict_label = np.vstack([
        val_predict_label[:, :11].argmax(1),
        val_predict_label[:, 11:22].argmax(1),
        val_predict_label[:, 22:33].argmax(1),
        val_predict_label[:, 33:44].argmax(1),
        val_predict_label[:, 44:55].argmax(1),
    ]).T
    val_label_pred = []
    for x in val_predict_label:
        val_label_pred.append(''.join(map(str, x[x!=10])))
    
    val_char_acc = np.mean(np.array(val_label_pred) == np.array(val_label))
    
    print('Epoch: {0}, Train loss: {1} \t Val loss: {2}'.format(epoch, train_loss, val_loss))
    print(val_char_acc)
    
    # 修改学习率，如果验证集的loss出现上升
    if epoch != 1 and val_loss > val_loss_last:
        loss_increase_num += 1
        if loss_increase_num == 2: # 出现上升两次
            loss_increase_num = 0
            if lr > 0.000011: # 不能过小
                lr = lr * 0.2
            print('learning rate:\t%f'%lr)
            optimizer = torch.optim.Adam(model.parameters(), lr, weight_decay=wd)

    val_loss_last = val_loss

    # 记录下验证集精度，保存两个，一个loss最优，一个acc最优
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), '../../data/Datawhale-Tianchi-CV_SVHN//model_v4_val_loss.pt')
    if val_char_acc > best_acc:
        best_acc = val_char_acc
        torch.save(model.state_dict(), '../../data/Datawhale-Tianchi-CV_SVHN//model_v4_val_acc.pt')

Epoch: 0, Train loss: 5.7430682307795475 	 Val loss: 6.090719919938308
0.0157
Epoch: 1, Train loss: 3.802279654302095 	 Val loss: 5.413386564988357
0.0155
Epoch: 2, Train loss: 3.559411726499859 	 Val loss: 5.102651119232178
0.0309
Epoch: 3, Train loss: 3.457689881324768 	 Val loss: 4.796212709867037
0.0884
Epoch: 4, Train loss: 3.441864120332818 	 Val loss: 4.969727002657377
0.0793
Epoch: 5, Train loss: 3.5030708375730013 	 Val loss: 5.3574768946721
0.0663
learning rate:	0.000200
Epoch: 6, Train loss: 2.8991821000450537 	 Val loss: 3.9920370211968055
0.3254
Epoch: 7, Train loss: 2.645596428921348 	 Val loss: 3.5559367583348203
0.3625
Epoch: 8, Train loss: 2.5710898637771606 	 Val loss: 3.6295955731318545
0.3685
Epoch: 9, Train loss: 2.5431840670736214 	 Val loss: 3.4849334496718187
0.3882
Epoch: 10, Train loss: 2.526869529171994 	 Val loss: 3.797806684787457
0.3737
learning rate:	0.000040
Epoch: 11, Train loss: 2.2980910978819193 	 Val loss: 2.979154109954834
0.4585
Epoch: 12, Train l

训练两个2 Epoch后，输出的训练日志为：
         
Epoch: 0, Train loss: 3.1 	 Val loss: 3.4 验证集精度：0.3439       
Epoch: 1, Train loss: 2.1 	 Val loss: 2.9 验证集精度：0.4346

* Quadro P1000: batch_size=40 epoch=2 需要6min15s GPU-util:50%-60%
* Quadro P1000: batch_size=200 epoch=2 需要4min59s GPU-util:60%-98%
* i7-8850H: batch_size=40 epoch=2 需要47min30s CPU-util:50%

In [7]:
torch.save(model.state_dict(), '../../data/Datawhale-Tianchi-CV_SVHN//model_v4_val_epoch20.pt')

## 步骤6：对测试集样本进行预测，生成提交文件          

In [ ]:
# # 保存的最好的模型载入
# model.load_state_dict(torch.load('../../data/Datawhale-Tianchi-CV_SVHN//model.pt'))

In [ ]:
test_path = glob.glob('../../data/Datawhale-Tianchi-CV_SVHN/mchar_test_a/*.png')
test_path.sort()
test_label = [[1]] * len(test_path)
print(len(val_path), len(val_label))

test_loader = torch.utils.data.DataLoader(
    SVHNDataset(test_path, test_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=0, # win下num_works改为0,linux=10
)

test_predict_label = predict(test_loader, model, 1)

test_label = [''.join(map(str, x)) for x in test_loader.dataset.img_label]
test_predict_label = np.vstack([
    test_predict_label[:, :11].argmax(1),
    test_predict_label[:, 11:22].argmax(1),
    test_predict_label[:, 22:33].argmax(1),
    test_predict_label[:, 33:44].argmax(1),
    test_predict_label[:, 44:55].argmax(1),
]).T

test_label_pred = []
for x in test_predict_label:
    test_label_pred.append(''.join(map(str, x[x!=10])))

In [ ]:
import pandas as pd
df_submit = pd.read_csv('../../data/Datawhale-Tianchi-CV_SVHN/mchar_sample_submit_A.csv')
df_submit['file_code'] = test_label_pred
df_submit.to_csv('../../data/Datawhale-Tianchi-CV_SVHN/resnet18_baseline_v1lr_epoch20.csv', index=None)

在训练完成2个Epoch后，模型在测试集上的成绩应该在0.33左右。